In [35]:
import torch
import torchvision

# Import necessary libraries
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt



In [36]:
# Step 1: Data Preparation
# Define transformations with augmentation for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define transformations for testing
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [37]:
# Step 2: Load CIFAR-10 dataset
# Training data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                        shuffle=True, num_workers=2)

# Testing data
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                       shuffle=False, num_workers=2)

# CIFAR-10 classes
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [38]:
# Step 3: Create CNN Model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # First convolutional layer: 3 input channels, 6 output channels, 5x5 kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        # Max pooling layer with 2x2 kernel
        self.pool = nn.MaxPool2d(2, 2)
        # Second convolutional layer: 6 input channels, 16 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5)
        # Fully connected layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Apply conv1, ReLU, and pooling
        x = self.pool(torch.relu(self.conv1(x)))
        # Apply conv2, ReLU, and pooling
        x = self.pool(torch.relu(self.conv2(x)))
        # Flatten the tensor for the fully connected layers
        x = x.view(-1, 16 * 5 * 5)
        # Apply fully connected layers with ReLU
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [39]:
# Step 4: Training Setup
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [40]:
# Step 5: Training Loop
def train_model(epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # Get the inputs and labels
            inputs, labels = data

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Print statistics
            running_loss += loss.item()
            if i % 200 == 199:    # Print every 200 mini-batches
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 200:.3f}')
                running_loss = 0.0

    print('Finished Training!')

In [41]:

# Step 6: Evaluation Function
def evaluate_model():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy on test images: {100 * correct / total}%')

In [42]:
# Step 7: Prediction
def predict_image(image):
    # Set the model to evaluation mode
    model.eval()
    with torch.no_grad():
        # Check if image is already a tensor
        if isinstance(image, torch.Tensor):
            # Add batch dimension if needed
            if image.dim() == 3:
                image = image.unsqueeze(0)
        else:
            # Transform the image if it's not a tensor
            image = transform_test(image).unsqueeze(0)

        output = model(image)
        _, predicted = torch.max(output.data, 1)
        return classes[predicted.item()]

In [43]:
# Step 8: Run Training and Evaluation
if __name__ == '__main__':
    print("Starting training...")
    train_model()
    print("Evaluating model...")
    evaluate_model()

    # Test the prediction function with a sample image from the test set
    sample_image, sample_label = testset[0]
    predicted_class = predict_image(sample_image)
    print(f'Predicted: {predicted_class}, Actual: {classes[sample_label]}')

Starting training...
[Epoch 1, Batch 200] Loss: 2.303
[Epoch 1, Batch 400] Loss: 2.301
[Epoch 1, Batch 600] Loss: 2.297
[Epoch 1, Batch 800] Loss: 2.288
[Epoch 1, Batch 1000] Loss: 2.253
[Epoch 1, Batch 1200] Loss: 2.155
[Epoch 1, Batch 1400] Loss: 2.055
[Epoch 2, Batch 200] Loss: 1.991
[Epoch 2, Batch 400] Loss: 1.951
[Epoch 2, Batch 600] Loss: 1.906
[Epoch 2, Batch 800] Loss: 1.850
[Epoch 2, Batch 1000] Loss: 1.813
[Epoch 2, Batch 1200] Loss: 1.775
[Epoch 2, Batch 1400] Loss: 1.721
[Epoch 3, Batch 200] Loss: 1.678
[Epoch 3, Batch 400] Loss: 1.644
[Epoch 3, Batch 600] Loss: 1.616
[Epoch 3, Batch 800] Loss: 1.609
[Epoch 3, Batch 1000] Loss: 1.586
[Epoch 3, Batch 1200] Loss: 1.551
[Epoch 3, Batch 1400] Loss: 1.587
[Epoch 4, Batch 200] Loss: 1.526
[Epoch 4, Batch 400] Loss: 1.498
[Epoch 4, Batch 600] Loss: 1.494
[Epoch 4, Batch 800] Loss: 1.492
[Epoch 4, Batch 1000] Loss: 1.458
[Epoch 4, Batch 1200] Loss: 1.455
[Epoch 4, Batch 1400] Loss: 1.447
[Epoch 5, Batch 200] Loss: 1.411
[Epoch 5, 